In [ ]:
import sys
sys.path.insert(0, '../')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ydata_profiling import ProfileReport
import io
import tabulate
import helpers_eda_inicial
import helpers_cbc
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [ ]:
personas = pd.read_csv('../../../assets/silver/data_utilizada/solo_datos_personales.csv')
actas_con_personas = pd.read_csv('../../../assets/silver/data_utilizada/personas_un_acta_por_fila.csv')

In [3]:
actas_con_personas['fecha'] = pd.to_datetime(actas_con_personas['fecha'], format='%Y-%m-%d')
actas_con_personas['dni'] = actas_con_personas['dni'].astype(str)
personas.drop(columns = ['Unnamed: 0'], inplace = True)
actas_con_personas.drop(columns = ['Unnamed: 0'], inplace = True)

In [ ]:
actas_con_personas.dtypes

In [ ]:
personas.dtypes

Tenemos que crear la variable target para cada uno de los DNIs

In [ ]:
actas_con_personas['dni'].nunique()

In [ ]:
actas_con_personas[~actas_con_personas['dni'].isin(personas['dni'].unique())]['dni'].unique()

In [265]:
personas_dni = personas['dni'].unique()
periodos = actas_con_personas['semestre_relativo'].dropna().unique()
combinaciones = pd.MultiIndex.from_product([personas_dni, periodos], names=['dni', 'semestre_relativo']).to_frame(index=False)

In [ ]:
personas_dni.size

In [ ]:
periodos

In [268]:
# Crear columnas auxiliares
actas_con_personas['es_inscripcion'] = (
    (actas_con_personas['tipo_acta'] == 'Acta de Regulares/Promociones')
)

actas_con_personas['es_tp_aprobado'] = (
    (actas_con_personas['tipo_acta'] == 'Acta de Regulares/Promociones') &
    (actas_con_personas['resultado'] == 'Aprobado')
)

actas_con_personas['es_final_inscripto'] = (
    actas_con_personas['tipo_acta'] == 'Acta de Examen'
)

# Agrupar y calcular métricas
resumen_v2 = actas_con_personas.groupby(['dni', 'semestre_relativo']).agg(
    inscripciones=('es_inscripcion', 'sum'),
    tp_aprobados=('es_tp_aprobado', 'sum'),
    finales_inscriptos=('es_final_inscripto', 'sum')
).reset_index()

# 5. Hacer el merge con el resumen
resumen_completo = combinaciones.merge(
    resumen_v2,
    on=['dni', 'semestre_relativo'],
    how='left'
).fillna(0)

In [ ]:
resumen_completo.groupby('semestre_relativo').count()

In [270]:
resumen_completo['total_actividad'] = (
    resumen_completo['inscripciones'] +
    resumen_completo['tp_aprobados'] +
    resumen_completo['finales_inscriptos']
)

In [ ]:
resumen_completo.head()

In [ ]:
resumen_completo.groupby('semestre_relativo').count()

In [ ]:
metricas = ['inscripciones', 'tp_aprobados', 'finales_inscriptos', 'total_actividad']

for metrica in metricas:
    tabla = pd.crosstab(resumen_completo['semestre_relativo'], resumen_completo[metrica])
    plt.figure(figsize=(10, 6))
    sns.heatmap(tabla, annot=True, fmt='d', cmap='Blues')
    plt.title(f'Distribución de {metrica} por Semestre Relativo')
    plt.xlabel(metrica)
    plt.ylabel('Semestre Relativo')
    plt.show()

In [ ]:
tabla = pd.crosstab(resumen_completo['semestre_relativo'], resumen_completo['total_actividad'])
plt.figure(figsize=(10, 6))
sns.heatmap(tabla, annot=True, fmt='d', cmap='Blues')
plt.title(f'Distribución de Total de Actividad por Semestre Relativo', fontsize=15)
plt.xlabel('Total de Actividad', fontsize=15)
plt.ylabel('Semestre Relativo', fontsize=15)
plt.xticks(fontsize=11)
plt.yticks(fontsize=13)
plt.show()

In [ ]:
y

In [ ]:
y_pred

In [ ]:
# X = índice (semestre_relativo), convertido en array 2D
X = tabla.index.to_numpy().reshape(-1, 1)
y = tabla[0.0].to_numpy()

# Ajustar regresión lineal
modelo = LinearRegression()
modelo.fit(X, y)
y_pred = modelo.predict(X)
mse = mean_squared_error(y, y_pred)
print(f"Error cuadrático medio (MSE): {mse:.4f}")

# Graficar
plt.figure(figsize=(8, 5))
sns.scatterplot(x=tabla.index, y=tabla[0.0], label='Datos reales')
plt.plot(tabla.index, y_pred, color='red', label='Regresión lineal')
plt.title(f'Cantidad de personas con total de actividad = 0 por semestre relativo', fontsize=15)
plt.xticks(range(0,5), fontsize=11)
plt.yticks(fontsize=11)
plt.xlabel('Semestre relativo', fontsize=14)
plt.ylabel('Cantidad', fontsize=14)
plt.grid(True)
plt.legend(fontsize=12, title_fontsize=13)
plt.tight_layout()
plt.show()

In [ ]:
# X = índice (semestre_relativo), convertido en array 2D
X = tabla.index.to_numpy().reshape(-1, 1)
y = tabla[1.0].to_numpy() + tabla[2.0].to_numpy() + tabla[3.0].to_numpy()
actividad_total = tabla[1.0] + tabla[2.0] + tabla[3.0]

# Ajustar regresión lineal
modelo = LinearRegression()
modelo.fit(X, y)
y_pred = modelo.predict(X)
mse = mean_squared_error(y, y_pred)
print(f"Error cuadrático medio (MSE): {mse:.4f}")

# Graficar
plt.figure(figsize=(8, 5))
sns.scatterplot(x=tabla.index, y=actividad_total, label='Datos reales')
plt.plot(tabla.index, y_pred, color='red', label='Regresión lineal')
plt.title(f'Cantidad de personas con total de actividad entre 1 y 3 por semestre relativo', fontsize=15)
plt.xticks(range(0,5), fontsize=11)
plt.yticks(fontsize=11)
plt.xlabel('Semestre relativo', fontsize=14)
plt.ylabel('Cantidad', fontsize=14)
plt.grid(True)
plt.legend(fontsize=12, title_fontsize=13)
plt.tight_layout()
plt.show()

In [279]:
#resumen_completo.to_csv('../../../../assets/silver/data_utilizada/resumen_personas_por_semestre.csv', index=False)

In [ ]:
resumen_completo_semestre_4 = resumen_completo[resumen_completo['semestre_relativo'] == 4]
resumen_completo_semestre_4['dni'].nunique()

In [ ]:
resumen_completo_semestre_4['target'] = np.where(resumen_completo_semestre_4['total_actividad'] < 3, 1, 0)
resumen_completo_semestre_4= resumen_completo_semestre_4[['dni','target']]

In [ ]:
resumen_completo_semestre_4['target'].value_counts(dropna= False)

In [ ]:
personas_con_target = pd.merge(personas, resumen_completo_semestre_4, on='dni', how='left')
personas_con_target['target'].value_counts(dropna= False)

In [ ]:
semestre_de_inscripcion = actas_con_personas[
    ['dni', 'es_cohorte_2022C1', 'es_cohorte_2022C2', 'es_cohorte_2021C1', 'es_cohorte_2021C2']
].drop_duplicates()
print(semestre_de_inscripcion['dni'].nunique())

# Contar cuántos True hay por fila
cohorte_cols = ['es_cohorte_2022C1', 'es_cohorte_2022C2', 'es_cohorte_2021C1', 'es_cohorte_2021C2']
semestre_de_inscripcion['cohortes_true_count'] = semestre_de_inscripcion[cohorte_cols].sum(axis=1)

# Filtrar los casos incorrectos (donde no hay exactamente 1 True)
casos_invalidos = semestre_de_inscripcion[semestre_de_inscripcion['cohortes_true_count'] != 1]
if casos_invalidos.empty:
    print("✔️ Todos los DNI tienen exactamente una cohorte asignada.")
else:
    print("❌ Hay DNIs con múltiples o ninguna cohorte asignada:")
    print(casos_invalidos)

In [285]:
cohorte_cols = ['es_cohorte_2022C1', 'es_cohorte_2022C2', 'es_cohorte_2021C1', 'es_cohorte_2021C2']
semestre_de_inscripcion['cohorte'] = semestre_de_inscripcion[cohorte_cols].idxmax(axis=1)
semestre_de_inscripcion['cohorte'] = semestre_de_inscripcion['cohorte'].str.replace('es_cohorte_', '')

In [286]:
semestre_de_inscripcion = semestre_de_inscripcion[['dni', 'cohorte']]

In [287]:
personas_con_target = pd.merge(personas_con_target, semestre_de_inscripcion, on='dni', how='left')

In [ ]:
personas_con_target.columns

In [ ]:
personas_con_target.groupby('cohorte')['target'].value_counts(dropna= False)

In [ ]:
# Obtener los conteos (incluyendo NaN)
conteos = personas_con_target['target'].value_counts(dropna=False).reset_index()
conteos.columns = ['target', 'count']

# Convertir NaN a string para que seaborn lo grafique
conteos['target'] = conteos['target'].astype(str)

# Crear barplot
plt.figure(figsize=(8, 5))
sns.barplot(data=conteos, x='target', y='count', palette='pastel')
plt.title('Distribución de variable Target', fontsize=15)
plt.xlabel('Target', fontsize=12)
plt.ylabel('Cantidad', fontsize=12)
plt.xticks(fontsize=11)
plt.yticks(fontsize=11)
plt.tight_layout()
plt.show()

In [ ]:
# Obtener los conteos (incluyendo NaN)
conteos = personas_con_target['target'].value_counts(dropna=False).reset_index()
conteos.columns = ['target', 'count']

# Calcular proporciones
total = conteos['count'].sum()
conteos['proporcion'] = conteos['count'] / total

# Reemplazar valores por etiquetas más descriptivas
mapeo_etiquetas = {0: 'No Abandono', 1: 'Abandono'}
conteos['target'] = conteos['target'].map(mapeo_etiquetas).fillna('Desconocido')

# Crear barplot con proporciones
plt.figure(figsize=(8, 5))
sns.barplot(data=conteos, x='target', y='proporcion', palette ={'No Abandono': "#5D81E2", 'Abandono': "#6CCDA3"})
plt.title('Distribución de variable Target', fontsize=15)
plt.xlabel('Target', fontsize=13)
plt.ylabel('Proporción', fontsize=12)
plt.xticks(fontsize=12)
plt.yticks(fontsize=11)
plt.tight_layout()
plt.show()

In [ ]:
conteos

In [ ]:
# Agrupar y contar
df_plot = (
    personas_con_target
    .groupby(['cohorte', 'target'])
    .size()
    .reset_index(name='count')
)

# Crear barplot
plt.figure(figsize=(10, 6))
sns.barplot(data=df_plot, x='cohorte', y='count', hue='target', palette ={0: "#5D81E2", 1: "#6CCDA3"})
plt.title('Distribución de Target por Cohorte')
plt.xlabel('Cohorte')
plt.ylabel('Cantidad')
plt.legend(title='Target')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Paso 1: Conteo de cada (cohorte, target)
conteos = (
    personas_con_target
    .groupby(['cohorte', 'target'])
    .size()
    .reset_index(name='count')
)

# Paso 2: Calcular proporciones por cohorte
conteos['proporcion'] = (
    conteos
    .groupby('cohorte')['count']
    .transform(lambda x: x / x.sum())
)

# Paso 3: Plotear
plt.figure(figsize=(10, 6))
sns.barplot(data=conteos, x='cohorte', y='proporcion', hue='target', palette ={0: "#5D81E2", 1: "#6CCDA3"})
plt.title('Proporción de Target por Cohorte')
plt.xlabel('Cohorte')
plt.ylabel('Proporción')
plt.legend(title='Target')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


## Sexo

In [ ]:
# Agrupar y contar
df_plot = (
    personas_con_target
    .groupby(['sexo', 'target'])
    .size()
    .reset_index(name='count')
)

# Crear barplot
plt.figure(figsize=(10, 6))
sns.barplot(data=df_plot, x='sexo', y='count', hue='target', palette ={0: "#5D81E2", 1: "#6CCDA3"})
plt.title('Distribución de Target por Sexo')
plt.xlabel('Cohorte')
plt.ylabel('Cantidad')
plt.legend(title='Target')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Paso 1: Conteo de cada (sexo, target)
conteos = (
    personas_con_target
    .groupby(['sexo', 'target'])
    .size()
    .reset_index(name='count')
)

# Paso 2: Calcular proporciones por sexo
conteos['proporcion'] = (
    conteos
    .groupby('sexo')['count']
    .transform(lambda x: x / x.sum())
)



# Paso 3: Plotear
plt.figure(figsize=(10, 6))
sns.barplot(data=conteos, x='sexo', y='proporcion', hue='target', palette ={0: "#5D81E2", 1: "#6CCDA3"})
plt.title('Proporción de Target por Sexo')
plt.xlabel('Sexo')
plt.ylabel('Proporción')
plt.legend(title='Target')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## Situación Laboral

In [ ]:
# Copiar dataframe y completar valores faltantes
df_plot = personas_con_target.copy()
df_plot['situacion_laboral'] = df_plot['situacion_laboral'].fillna('Faltante')

# Reemplazar nombres largos por versiones más legibles
nombres_largos = {
    'No trabajó y no buscó trabajo (no esta pensando en trabajar)': 'No trabaja ni busca trabajo',
    'Trabajó al menos una hora (incluye a los que no trabajaron por licencia, vacaciones, enfermedad)': 'Trabajó al menos una hora',
    'No trabajó y buscó trabajo en algún momento de los últimos 30 días': 'No trabaja pero busca',
    'Desconoce': 'Desconoce'
}
df_plot['situacion_laboral'] = df_plot['situacion_laboral'].replace(nombres_largos)

# Agrupar por situación laboral y target, y contar ocurrencias
df_plot = (
    df_plot
    .groupby(['situacion_laboral', 'target'])
    .size()
    .reset_index(name='count')
)

mapeo_etiquetas = {0: 'No Abandono', 1: 'Abandono'}
df_plot['target'] = df_plot['target'].map(mapeo_etiquetas).fillna('Desconocido')

# Obtener orden de categorías según cantidad total (de más a menos)
orden_situacion = (
    df_plot
    .groupby('situacion_laboral')['count']
    .sum()
    .sort_values(ascending=False)
    .index
)

# Crear gráfico de barras
plt.figure(figsize=(8, 6))
sns.barplot(
    data=df_plot,
    x='situacion_laboral',
    y='count',
    hue='target',
    palette ={'No Abandono': "#5D81E2", 'Abandono': "#6CCDA3"},
    order=orden_situacion
)
plt.title('Distribución de Target por Situación Laboral', fontsize=15)
plt.xlabel('Situación Laboral', fontsize=15)
plt.ylabel('Cantidad', fontsize=12)
plt.legend(title='Target', fontsize=13, title_fontsize=13)
plt.xticks(rotation=90, fontsize=11)
plt.yticks(fontsize=11)
plt.tight_layout()
plt.show()


In [ ]:
df_plot

In [ ]:
# Copiar dataframe y completar valores faltantes
df_plot = personas_con_target.copy()
df_plot['situacion_laboral'] = df_plot['situacion_laboral'].fillna('Faltante')

# Reemplazar nombres largos por versiones más legibles
nombres_largos = {
    'No trabajó y no buscó trabajo (no esta pensando en trabajar)': 'No trabaja ni busca trabajo',
    'Trabajó al menos una hora (incluye a los que no trabajaron por licencia, vacaciones, enfermedad)': 'Trabajó al menos una hora',
    'No trabajó y buscó trabajo en algún momento de los últimos 30 días': 'No trabaja pero busca',
    'Desconoce': 'Desconoce'
}
df_plot['situacion_laboral'] = df_plot['situacion_laboral'].replace(nombres_largos)

# Paso 1: Conteo de cada (situacion_laboral, target)
conteos = (
    df_plot
    .groupby(['situacion_laboral', 'target'])
    .size()
    .reset_index(name='count')
)

# Paso 2: Calcular proporciones por situacion_laboral
conteos['proporcion'] = (
    conteos
    .groupby('situacion_laboral')['count']
    .transform(lambda x: x / x.sum())
)

# Ordenar según total de personas en cada situación laboral (de mayor a menor)
orden_situacion = (
    conteos
    .groupby('situacion_laboral')['count']
    .sum()
    .sort_values(ascending=False)
    .index
)

# Paso 3: Plotear
plt.figure(figsize=(8, 6))
sns.barplot(
    data=conteos,
    x='situacion_laboral',
    y='proporcion',
    hue='target',
    order=orden_situacion,
    palette ={0: "#5D81E2", 1: "#6CCDA3"}
)
plt.title('Proporción de Target por Situación Laboral', fontsize=15)
plt.xlabel('Situación Laboral', fontsize=15)
plt.ylabel('Proporción', fontsize=12)
plt.legend(title='Target', fontsize=13, title_fontsize=13)
plt.xticks(rotation=90, fontsize=11)
plt.yticks(fontsize=11)
plt.tight_layout()
plt.show()


In [ ]:
# Filtrar solo los casos con target == 1
conteos_filtrado = conteos[conteos['target'] == 1].copy()

# Cambiar valor de target a 'Abandono' para el gráfico
conteos_filtrado['target'] = 'Abandono'

# Plotear
plt.figure(figsize=(8, 6))
sns.barplot(
    data=conteos_filtrado,
    x='situacion_laboral',
    y='proporcion',
    hue='target',  # opcional, ahora todo será "Abandono"
    order=orden_situacion,
    palette = {'No Abandono': "#5D81E2", 'Abandono': "#6CCDA3"}  # Verde
)
plt.title('Proporción de Abandono por Situación Laboral', fontsize=15)
plt.xlabel('Situación Laboral', fontsize=15)
plt.ylabel('Proporción', fontsize=12)
plt.legend(title='', fontsize=13)  # si prefieres eliminar completamente la leyenda, usa: plt.legend().remove()
plt.xticks(rotation=90, fontsize=11)
plt.yticks(fontsize=11)
plt.tight_layout()
plt.show()

In [ ]:
conteos_filtrado

## Acceso a internet

In [ ]:
# Agrupar y contar
df_plot = (
    personas_con_target
    .groupby(['acceso_a_internet', 'target'])
    .size()
    .reset_index(name='count')
)

# Crear barplot
plt.figure(figsize=(15, 6))
sns.barplot(data=df_plot, x='acceso_a_internet', y='count', hue='target')
plt.title('Distribución de Target por Acceso a internet')
plt.xlabel('Acceso a internet')
plt.ylabel('Cantidad')
plt.legend(title='Target')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

In [ ]:
# Paso 1: Conteo de cada (acceso_a_internet, target)
conteos = (
    personas_con_target
    .groupby(['acceso_a_internet', 'target'])
    .size()
    .reset_index(name='count')
)

# Paso 2: Calcular proporciones por acceso_a_internet
conteos['proporcion'] = (
    conteos
    .groupby('acceso_a_internet')['count']
    .transform(lambda x: x / x.sum())
)

# Paso 3: Plotear
plt.figure(figsize=(10, 6))
sns.barplot(data=conteos, x='acceso_a_internet', y='proporcion', hue='target')
plt.title('Proporción de Target por Acceso Internet')
plt.xlabel('Acceso Internet')
plt.ylabel('Proporción')
plt.legend(title='Target')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

## Carrera Principal recalculada

In [337]:
# Paso 1: Contar las veces que cada persona se inscribió a cada carrera
conteos = actas_con_personas.groupby(['dni', 'carrera']).size().reset_index(name='conteo')

# Paso 2: Para cada dni, quedarte con la carrera de mayor conteo
conteos = conteos.sort_values(['dni', 'conteo'], ascending=[True, False])
carrera_mas_frecuente = conteos.drop_duplicates(subset='dni', keep='first')
carrera_mas_frecuente
personas_con_target = pd.merge(personas_con_target, carrera_mas_frecuente[['dni', 'carrera']], on='dni', how='left')

In [ ]:
personas_con_target.columns

In [ ]:
# Agrupar y contar
df_plot = (
    personas_con_target
    .groupby(['carrera', 'target'])
    .size()
    .reset_index(name='count')
)

# Crear barplot
plt.figure(figsize=(15, 5))
sns.barplot(data=df_plot, x='carrera', y='count', hue='target')
plt.title('Distribución de Target por carrera principal')
plt.xlabel('Carrera principal')
plt.ylabel('Cantidad')
plt.legend(title='Target')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

In [ ]:
# Lista de carreras a mantener
carreras_mantener = [
    'Licenciatura en Ciencias de la Computación',
    'Licenciatura en Ciencias Biológicas',
    'Licenciatura en Ciencias Físicas',
    'Licenciatura en Ciencias de Datos',
    'Licenciatura en Ciencias Químicas',
    'Licenciatura en Ciencias Matemáticas',
    'Licenciatura en Ciencias de la Atmósfera',
    'Licenciatura en Ciencia y Tecnología de Alimentos',
    'Licenciatura en Ciencias Geológicas'
]

# Etiquetas personalizadas
etiquetas_personalizadas = {
    'Licenciatura en Ciencias de la Computación': 'Computación',
    'Licenciatura en Ciencias Biológicas': 'Biología',
    'Licenciatura en Ciencias Físicas': 'Física',
    'Licenciatura en Ciencias de Datos': 'Datos',
    'Licenciatura en Ciencias Químicas': 'Química',
    'Licenciatura en Ciencias Matemáticas': 'Matemática',
    'Licenciatura en Ciencias de la Atmósfera': 'Atmósfera',
    'Licenciatura en Ciencia y Tecnología de Alimentos': 'Alimentos',
    'Licenciatura en Ciencias Geológicas': 'Geología',
    'Otras carreras': 'Otras carreras'
}

# Crear columna agrupada
personas_con_target['carrera_filtrada'] = personas_con_target['carrera'].apply(
    lambda x: x if x in carreras_mantener else 'Otras carreras'
)

# Agrupar y contar
df_plot = (
    personas_con_target
    .groupby(['carrera_filtrada', 'target'])
    .size()
    .reset_index(name='count')
)

# Reemplazar nombres por etiquetas personalizadas
df_plot['carrera_filtrada'] = df_plot['carrera_filtrada'].replace(etiquetas_personalizadas)

# Ordenar categorías en el gráfico
orden_etiquetas = [etiquetas_personalizadas[c] for c in carreras_mantener] + ['Otras carreras']
df_plot['carrera_filtrada'] = pd.Categorical(df_plot['carrera_filtrada'], categories=orden_etiquetas, ordered=True)

mapeo_etiquetas = {0: 'No Abandono', 1: 'Abandono'}
df_plot['target'] = df_plot['target'].map(mapeo_etiquetas).fillna('Desconocido')

# Crear barplot
plt.figure(figsize=(9, 6))
sns.barplot(data=df_plot, x='carrera_filtrada', y='count', hue='target', palette = {'No Abandono': "#5D81E2", 'Abandono': "#6CCDA3"})
plt.title('Distribución de Target por carrera principal', fontsize=15)
plt.xlabel('Carrera principal', fontsize=15)
plt.ylabel('Cantidad', fontsize=15)
plt.legend(title='Target', fontsize=13, title_fontsize=13)
plt.xticks(rotation=90, fontsize=11)
plt.yticks(fontsize=11)
plt.tight_layout()
plt.show()


In [ ]:
# Lista de carreras a mantener
carreras_mantener = [
    'Licenciatura en Ciencias de la Computación',
    'Licenciatura en Ciencias Biológicas',
    'Licenciatura en Ciencias Físicas',
    'Licenciatura en Ciencias de Datos',
    'Licenciatura en Ciencias Químicas',
    'Licenciatura en Ciencias Matemáticas',
    'Licenciatura en Ciencias de la Atmósfera',
    'Licenciatura en Ciencia y Tecnología de Alimentos',
    'Licenciatura en Ciencias Geológicas'
]

# Etiquetas personalizadas
etiquetas_personalizadas = {
    'Licenciatura en Ciencias de la Computación': 'Computación',
    'Licenciatura en Ciencias Biológicas': 'Biología',
    'Licenciatura en Ciencias Físicas': 'Física',
    'Licenciatura en Ciencias de Datos': 'Datos',
    'Licenciatura en Ciencias Químicas': 'Química',
    'Licenciatura en Ciencias Matemáticas': 'Matemática',
    'Licenciatura en Ciencias de la Atmósfera': 'Atmósfera',
    'Licenciatura en Ciencia y Tecnología de Alimentos': 'Alimentos',
    'Licenciatura en Ciencias Geológicas': 'Geología',
    'Otras carreras': 'Otras carreras'
}

# Paso 1: Agrupar las carreras no deseadas como 'Otras carreras'
personas_con_target['carrera_filtrada'] = personas_con_target['carrera'].apply(
    lambda x: x if x in carreras_mantener else 'Otras carreras'
)

# Paso 2: Conteo por carrera filtrada y target
conteos = (
    personas_con_target
    .groupby(['carrera_filtrada', 'target'])
    .size()
    .reset_index(name='count')
)

# Paso 3: Calcular proporciones por carrera
conteos['proporcion'] = (
    conteos
    .groupby('carrera_filtrada')['count']
    .transform(lambda x: x / x.sum())
)

# Paso 4: Reemplazar etiquetas
conteos['carrera_filtrada'] = conteos['carrera_filtrada'].replace(etiquetas_personalizadas)

# Ordenar las etiquetas
orden_etiquetas = [etiquetas_personalizadas[c] for c in carreras_mantener] + ['Otras carreras']
conteos['carrera_filtrada'] = pd.Categorical(conteos['carrera_filtrada'], categories=orden_etiquetas, ordered=True)

# Filtrar solo los casos con target == 1
conteos_filtrado = conteos[conteos['target'] == 1].copy()

# Cambiar valor de target a 'Abandono' para el gráfico
conteos_filtrado['target'] = 'Abandono'

# Paso 5: Plotear
plt.figure(figsize=(9, 6))
sns.barplot(data=conteos_filtrado, x='carrera_filtrada', y='proporcion', hue='target', palette = {'No Abandono': "#5D81E2", 'Abandono': "#6CCDA3"})
plt.title('Proporción de Abandono por carrera principal', fontsize=15)
plt.xlabel('Carrera principal', fontsize=15)
plt.ylabel('Proporción', fontsize=15)
plt.legend(fontsize=13, title_fontsize=13)
plt.xticks(rotation=90, fontsize=11)
plt.yticks(fontsize=11)
plt.tight_layout()
plt.show()

In [ ]:
conteos_filtrado

In [ ]:
# Paso 1: Conteo de cada (carrera, target)
conteos = (
    personas_con_target
    .groupby(['carrera', 'target'])
    .size()
    .reset_index(name='count')
)

# Paso 2: Calcular proporciones por carrera
conteos['proporcion'] = (
    conteos
    .groupby('carrera')['count']
    .transform(lambda x: x / x.sum())
)

# Paso 3: Plotear
plt.figure(figsize=(15, 5))
sns.barplot(data=conteos, x='carrera', y='proporcion', hue='target')
plt.title('Proporción de Target por carrera principal')
plt.xlabel('Carrera principal')
plt.ylabel('Proporción')
plt.legend(title='Target')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

## Nivel estudio Madre

In [ ]:
# Reemplazar NaN por 'Faltante'
df_plot = personas_con_target.copy()
df_plot['nivel_estudio_madre'] = df_plot['nivel_estudio_madre'].fillna('Faltante')

# Agrupar y contar
df_plot = (
    df_plot
    .groupby(['nivel_estudio_madre', 'target'])
    .size()
    .reset_index(name='count')
)

# Ordenar categorías de mayor a menor según el total de personas
orden_estudio = (
    df_plot
    .groupby('nivel_estudio_madre')['count']
    .sum()
    .sort_values(ascending=False)
    .index
)

mapeo_etiquetas = {0: 'No Abandono', 1: 'Abandono'}
df_plot['target'] = df_plot['target'].map(mapeo_etiquetas).fillna('Desconocido')

# Crear barplot
plt.figure(figsize=(9, 6))
sns.barplot(
    data=df_plot,
    x='nivel_estudio_madre',
    y='count',
    hue='target',
    order=orden_estudio,
    palette = {'No Abandono': "#5D81E2", 'Abandono': "#6CCDA3"}
)
plt.title('Distribución de Target por Nivel de Estudio de la Madre', fontsize=14)
plt.xlabel('Nivel de estudio madre', fontsize=14)
plt.ylabel('Cantidad', fontsize=14)
plt.legend(title='Target', fontsize=13, title_fontsize=13)
plt.xticks(rotation=90, fontsize=11)
plt.tight_layout()
plt.show()


In [ ]:
# Reemplazar NaN por 'Faltante'
df_prop = personas_con_target.copy()
df_prop['nivel_estudio_madre'] = df_prop['nivel_estudio_madre'].fillna('Faltante')

# Paso 1: Conteo de cada (nivel_estudio_madre, target)
conteos = (
    df_prop
    .groupby(['nivel_estudio_madre', 'target'])
    .size()
    .reset_index(name='count')
)

# Paso 2: Calcular proporciones por nivel_estudio_madre
conteos['proporcion'] = (
    conteos
    .groupby('nivel_estudio_madre')['count']
    .transform(lambda x: x / x.sum())
)

# Paso 2b: Obtener orden según cantidad total (como en el gráfico de cantidades)
orden_estudio = (
    conteos
    .groupby('nivel_estudio_madre')['count']
    .sum()
    .sort_values(ascending=False)
    .index
)

# Filtrar solo los casos con target == 1
conteos_filtrado = conteos[conteos['target'] == 1].copy()

# Cambiar valor de target a 'Abandono' para el gráfico
conteos_filtrado['target'] = 'Abandono'

# Paso 3: Plotear
plt.figure(figsize=(9, 6))
sns.barplot(
    data=conteos_filtrado,
    x='nivel_estudio_madre',
    y='proporcion',
    hue='target',
    order=orden_estudio,
    palette = {'No Abandono': "#5D81E2", 'Abandono': "#6CCDA3"}
)
plt.title('Proporción de Abandono por Nivel de Estudio de la Madre', fontsize=14)
plt.xlabel('Nivel de estudio madre', fontsize=14)
plt.ylabel('Proporción', fontsize=14)
plt.legend(fontsize=13, title_fontsize=13)
plt.xticks(rotation=90, fontsize=11)
plt.yticks(fontsize=11)
plt.tight_layout()
plt.show()


In [ ]:
conteos_filtrado

## Nivel estudio Padre

In [ ]:
df_plot = personas_con_target.copy()
df_plot['nivel_estudio_padre'] = df_plot['nivel_estudio_padre'].fillna('Faltante')
# Agrupar y contar
df_plot = (
    df_plot
    .groupby(['nivel_estudio_padre', 'target'])
    .size()
    .reset_index(name='count')
)

# Ordenar categorías de mayor a menor según el total de personas
orden_estudio = (
    df_plot
    .groupby('nivel_estudio_padre')['count']
    .sum()
    .sort_values(ascending=False)
    .index
)

mapeo_etiquetas = {0: 'No Abandono', 1: 'Abandono'}
df_plot['target'] = df_plot['target'].map(mapeo_etiquetas).fillna('Desconocido')

# Crear barplot
plt.figure(figsize=(9, 6))
sns.barplot(data=df_plot, x='nivel_estudio_padre', y='count', hue='target', order=orden_estudio, palette = {'No Abandono': "#5D81E2", 'Abandono': "#6CCDA3"})
plt.title('Distribución de Target por Nivel Estudio del Padre', fontsize=14)
plt.xlabel('Nivel de estudio padre', fontsize=14)
plt.ylabel('Cantidad', fontsize=14)
plt.legend(title='Target', fontsize=13, title_fontsize=13)
plt.xticks(rotation=90, fontsize=11)
plt.yticks(fontsize=11)
plt.tight_layout()
plt.show()

In [ ]:
# Reemplazar NaN por 'Faltante'
df_prop = personas_con_target.copy()
df_prop['nivel_estudio_padre'] = df_prop['nivel_estudio_padre'].fillna('Faltante')

# Paso 1: Conteo de cada (nivel_estudio_padre, target)
conteos = (
    df_prop
    .groupby(['nivel_estudio_padre', 'target'])
    .size()
    .reset_index(name='count')
)

# Paso 2: Calcular proporciones por nivel_estudio_padre
conteos['proporcion'] = (
    conteos
    .groupby('nivel_estudio_padre')['count']
    .transform(lambda x: x / x.sum())
)

orden_estudio = (
    conteos
    .groupby('nivel_estudio_padre')['count']
    .sum()
    .sort_values(ascending=False)
    .index
)

# Filtrar solo los casos con target == 1
conteos_filtrado = conteos[conteos['target'] == 1].copy()

# Cambiar valor de target a 'Abandono' para el gráfico
conteos_filtrado['target'] = 'Abandono'

# Paso 3: Plotear
plt.figure(figsize=(9, 6))
sns.barplot(data=conteos_filtrado, x='nivel_estudio_padre', y='proporcion', hue='target', order=orden_estudio, palette = {'No Abandono': "#5D81E2", 'Abandono': "#6CCDA3"})
plt.title('Proporción de Abandono por Nivel de Estudio del Padre', fontsize=14)
plt.xlabel('Nivel de estudio padre', fontsize=14)
plt.ylabel('Proporción', fontsize=14)
plt.legend(fontsize=13, title_fontsize=13)
plt.xticks(rotation=90, fontsize=11)
plt.yticks(fontsize=11)
plt.tight_layout()
plt.show()

In [ ]:
conteos_filtrado

## Estado Madre

In [ ]:
# Agrupar y contar
df_plot = (
    personas_con_target
    .groupby(['situacion_madre', 'target'])
    .size()
    .reset_index(name='count')
)

# Crear barplot
plt.figure(figsize=(15, 6))
sns.barplot(data=df_plot, x='situacion_madre', y='count', hue='target')
plt.title('Distribución por situacion_madre')
plt.xlabel('Situación de la madre')
plt.ylabel('Cantidad')
plt.legend(title='Target')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

In [ ]:
# Paso 1: Conteo de cada (situacion_madre, target)
conteos = (
    personas_con_target
    .groupby(['situacion_madre', 'target'])
    .size()
    .reset_index(name='count')
)

# Paso 2: Calcular proporciones por situacion_madre
conteos['proporcion'] = (
    conteos
    .groupby('situacion_madre')['count']
    .transform(lambda x: x / x.sum())
)

# Paso 3: Plotear
plt.figure(figsize=(15, 6))
sns.barplot(data=conteos, x='situacion_madre', y='proporcion', hue='target')
plt.title('Proporción de Target por situacion_madre')
plt.xlabel('Situación de la madre')
plt.ylabel('Proporción')
plt.legend(title='Target')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

In [ ]:
# Agrupar y contar
df_plot = (
    personas_con_target
    .groupby(['situacion_padre', 'target'])
    .size()
    .reset_index(name='count')
)

# Crear barplot
plt.figure(figsize=(15, 6))
sns.barplot(data=df_plot, x='situacion_padre', y='count', hue='target')
plt.title('Distribución por situacion_padre')
plt.xlabel('Situación de la padre')
plt.ylabel('Cantidad')
plt.legend(title='Target')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

In [ ]:
# Paso 1: Conteo de cada (situacion_padre, target)
conteos = (
    personas_con_target
    .groupby(['situacion_padre', 'target'])
    .size()
    .reset_index(name='count')
)

# Paso 2: Calcular proporciones por situacion_padre
conteos['proporcion'] = (
    conteos
    .groupby('situacion_padre')['count']
    .transform(lambda x: x / x.sum())
)

# Paso 3: Plotear
plt.figure(figsize=(15, 6))
sns.barplot(data=conteos, x='situacion_padre', y='proporcion', hue='target')
plt.title('Proporción de Target por situacion_padre')
plt.xlabel('Situación de la padre')
plt.ylabel('Proporción')
plt.legend(title='Target')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

## Edad

In [ ]:
personas_con_target['edad_inscripcion'].describe()

In [ ]:
personas_con_target['grupo_edad'] = pd.cut(personas_con_target['edad_inscripcion'], bins=[16, 20, 30, 64])

# Normalizado (proporciones)
prop_df = personas_con_target.groupby('grupo_edad')['target'].value_counts(normalize=True).unstack()

prop_df.plot(kind='bar', stacked=False)
plt.title("Distribución del target por grupo etario")
plt.ylabel("Proporción")
plt.xlabel("Grupo etario")
plt.xticks(rotation=0)
plt.legend(title="Target")
plt.show()

In [ ]:
# Calcular los conteos de cada valor de target por grupo etario
conteos_df = personas_con_target.groupby('grupo_edad')['target'].value_counts().unstack(fill_value=0)

# Graficar como barras apiladas
conteos_df.plot(kind='bar', stacked=False)
plt.title("Cantidad de personas por grupo etario y target")
plt.ylabel("Cantidad")
plt.xlabel("Grupo etario")
plt.legend(title="Target")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
personas_con_target['grupo_edad'] = pd.cut(personas_con_target['edad_inscripcion'], bins=[16, 18, 20, 30, 64])

# Normalizado (proporciones)
prop_df = personas_con_target['grupo_edad'].value_counts(normalize=False, sort=False)

prop_df.plot(kind='bar', stacked=False)
plt.title("Distribución por grupo etario", fontsize=15)
plt.ylabel("Proporción", fontsize=15)
plt.xlabel("Grupo etario", fontsize=15)
plt.xticks(rotation=0, fontsize=13)
plt.yticks(fontsize=13)
plt.show()

In [ ]:
prop_df

In [ ]:
personas_con_target['grupo_edad'] = pd.cut(personas_con_target['edad_inscripcion'], bins=[16, 18, 20, 30, 64])

# Normalizado (proporciones)
prop_df = personas_con_target['grupo_edad'].value_counts(normalize=True, sort=False)

prop_df.plot(kind='bar', stacked=False)
plt.title("Distribución por grupo etario", fontsize=15)
plt.ylabel("Proporción", fontsize=15)
plt.xlabel("Grupo etario", fontsize=15)
plt.xticks(rotation=0, fontsize=13)
plt.yticks(fontsize=13)
plt.show()

In [ ]:
# Renombrar columnas del DataFrame
personas_con_target['grupo_edad'] = pd.cut(personas_con_target['edad_inscripcion'], bins=[16, 18, 20, 30, 64])

# Normalizado (proporciones)
prop_df = personas_con_target.groupby('grupo_edad')['target'].value_counts(normalize=True).unstack()
prop_df.rename(columns={0: 'No Abandono', 1: 'Abandono'}, inplace=True)

prop_df.plot(kind='bar', stacked=False)
plt.title("Distribución del target por grupo etario", fontsize=15)
plt.ylabel("Proporción", fontsize=15)
plt.xlabel("Grupo etario", fontsize=15)
plt.xticks(rotation=0, fontsize=13)
plt.yticks(fontsize=13)
plt.legend(title="Target", fontsize=13, title_fontsize=13)
plt.show()

In [ ]:
# Calcular los conteos de cada valor de target por grupo etario
#conteos_df = personas_con_target.groupby('grupo_edad')['target'].value_counts().unstack(fill_value=0)

# Normalizado (proporciones)
conteos_df = personas_con_target.groupby('grupo_edad')['target'].value_counts().unstack()
conteos_df.rename(columns={0: 'No Abandono', 1: 'Abandono'}, inplace=True)

# Graficar como barras apiladas
conteos_df.plot(kind='bar', stacked=False, color=["#5D81E2", "#6CCDA3"])
plt.title("Cantidad de Personas por Grupo Etario y Valor de Target", fontsize=15)
plt.ylabel("Cantidad", fontsize=15)
plt.xlabel("Grupo etario", fontsize=15)
plt.legend(title="Target", fontsize=13, title_fontsize=13, loc='upper center')
plt.xticks(rotation=0, fontsize=13)
plt.yticks(fontsize=13)
plt.tight_layout()
plt.show()

In [ ]:
# Crear grupos
personas_con_target['grupo_edad'] = pd.cut(personas_con_target['edad_inscripcion'], bins=[16, 18, 20, 30, 64])

# Proporciones por grupo
prop_df = personas_con_target.groupby('grupo_edad')['target'].value_counts(normalize=True).unstack()

# Proporción de target=1
prop_target1 = prop_df[1].values
prop_df.rename(columns={0: 'No Abandono', 1: 'Abandono'}, inplace=True)
prop_df_abandono = prop_df['Abandono'] 

# Posiciones X: usar las posiciones enteras como en el gráfico de barras
x_pos = np.arange(len(prop_target1)).reshape(-1, 1)

# Ajustar regresión lineal
reg = LinearRegression().fit(x_pos, prop_target1)
y_pred = reg.predict(x_pos)

mse = mean_squared_error(prop_target1, y_pred)
print(f"Error cuadrático medio (MSE): {mse:.4f}")

# Graficar barras
ax = prop_df_abandono.plot(kind='bar', stacked=False, color=["#6CCDA3"])
plt.plot(x_pos, y_pred, color='black', linewidth=2.5, label='Reg. Lineal target=Abandono')

# Títulos y estilos
plt.title("Proporción de Abandono por Grupo Etario", fontsize=15)
plt.ylabel("Proporción", fontsize=15)
plt.xlabel("Grupo etario", fontsize=15)
plt.xticks(rotation=0, fontsize=13)
plt.yticks(fontsize=13)
plt.legend(fontsize=12, title_fontsize=13, loc='upper center')
plt.grid(axis='y', linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()


In [ ]:
x_pos

In [ ]:
prop_df_abandono

In [ ]:
#personas_con_target.to_csv('../../../../assets/silver/data_utilizada/personas_con_target.csv', index= False)